In [2]:
import numpy as np
import matplotlib.pyplot as plt
from matplotlib import animation, rc
# some handy functions to use along widgets
from IPython.display import display, Markdown, clear_output, HTML
from ipywidgets import Layout
# widget packages
import ipywidgets as widgets
import time

In [3]:
class Chromatograph:
    def __init__(self,A=5,B=95,epsilon=[1000,1000,1000],initialValues=[100,100,100],flowRate = 1, runTime=10):
        self.epsilon = epsilon #molar absorptivities
        self.flowRate = float(flowRate)
        self.initialValues = initialValues
        self.runTime = runTime
        self.A = A # water %
        self.B = B # methanol %
        self.reset()
    
    # initialize the simulation (and reset time to t=0)
    def reset(self):
        self.numComponents = 3
        self.step = 0
        self.K = self.calcK(self.A,self.B)
        # number of plates in the column.  Set so that a 1 mL/min flow rate gives 100 plates
        self.numPlates = int(100/float(self.flowRate))
        # number of steps the simulation will take at 10 steps / minute (100 steps in 10 minutes)
        self.maxSteps = int(self.runTime*10)
        # initialize the mobile phase and stationary phase arrays to hold the amount of each component
        self.mobilePhase = np.zeros((self.numPlates,self.numComponents))
        self.stationaryPhase = np.zeros((self.numPlates,self.numComponents))
        self.mobilePhase[0] = self.initialValues  #inject into mobile phase at t=0
        # detector array to hold the HPLC output
        self.detector = np.zeros((self.maxSteps,self.numComponents))
        
        # min and max of compound amounts, not used anymore.
        self.vmin = 0
        self.vmax = np.sum(self.initialValues)
        
        # calculate the partition coefficient for each compound 1, 2, and 3
        # initial K values were set to vaguely reproduce past class data.
        # K values are adjusted based on the polarity of water vs methanol
        # then fudge factors are added to scale the simulation to 10 "minutes"
        
    def calcK (self,A,B):
        P = {'water': 10.2, 'methanol': 5.1} 
        # higher P = higher K
        # aspirin 7.7 minutes
        # acetaminophen 4.4 minutes
        # caffeine 16 minutes
        Peff = (P['water'] * A + P['methanol'] * B) / 100.0
        Kmult = (10-(Peff-5.1)*9.9/5.1)*0.4
        K1 = 7.7/4.4 * Kmult
        K2 = 1.0 * Kmult
        K3 = 16.0/4.4 * Kmult
        return [K1,K2,K3]
    
    def printSettings(self):
        print (f"Run Time: {self.runTime} minutes")
        print (f"Flow Rate: {self.flowRate} mL/min")
        print (f"A:  {self.A}%   B:  {self.B}%")
        
        # advance the time step for the separation by steps
    def nextStep(self,steps):
        for step in range(0,steps):
            # if the step is longer than the detector array we allocated an array for then extend it
            if self.step >= self.maxSteps:
                newDetector = np.zeros((self.maxSteps,self.numComponents))
                self.detector = np.append(self.detector,newDetector,axis=0)
                self.maxSteps += self.maxSteps
                
            # move the mobile phase 1 plate to the right (closer to the detector)   
            for plate in range(self.numPlates-1,0,-1):
                self.mobilePhase[plate] = self.mobilePhase[plate-1]
                # this just makes sure the mobilePhase moves rather than duplicates itself on the left side
                self.mobilePhase[plate-1] = 0
                    
            # any compound at the end of the column comes off to the detector
            # perhaps this should go above the previous for loop
            for component in range(0,self.numComponents):
                self.detector[self.step][component] = self.epsilon[component] * self.mobilePhase[-1][component]

                for plate in range(0,self.numPlates):    
                    # partition between stationary phase / mobile phase
                    # the fraction in the stationary phase will be K / (K+1)
                    # the fraction in the mobile phase will be 1 - (K/(K+1))
                    Xs=self.stationaryPhase[plate][component]
                    Xm=self.mobilePhase[plate][component]
                    totalX = Xs + Xm
                    
                    fs = self.K[component]/(self.K[component]+1)
                    fm = 1-fs
                    
                    newXs = totalX*fs
                    newXm = totalX*fm
                    
                    self.stationaryPhase[plate][component] = newXs
                    self.mobilePhase[plate][component] = newXm
            
            #advance the timestep
            self.step += 1
            
    #return the amount of component in the mobile phase of each plate of the column
    def getMobilePhase(self,component):
        return self.mobilePhase[:,component]
    
    # same for the stationary phase
    def getStationaryPhase(self,component):
        return self.stationaryPhase[:,component]
    
    # return the detector data.  Have to format it properly for graphing
    def getDetectorData(self,component):
        return [np.arange(0,self.maxSteps)/100,self.detector[:,component]]
    
    # return all the mobile phase components formatted for graphing
    def getData(self,component):
        return [np.arange(0,self.numPlates),self.getMobilePhase(component)]
    
    # the max concentration of everything in the mobile phase
    # used to set the y-axis max value.
    # I was going to use this to set the max scale for color map too, but it didn't work well
    # right now the color map auto-scales, so the colors change as components spread and come off the column
    def maxMobilePhase(self):
        return np.amax(self.mobilePhase)
    
    # max amount in the detector for scaling
    def maxDetector(self):
        return np.amax(self.detector)
    
    # print out the peak integral table. It uses the actual amount of component so it's perfect even if
    # the components overlap
    # If I had more time I might sum the mAU for each component to get total absorbance and then autointegrate
    # It scales by a random factor between 0.98 - 1.02 so that data isn't perfect.
    
    def printPeakTable(self):
        print (f'{"time / min":^20}{"peak height / mAU":^20}{"peak area / mAU*min":^20}')
        peaks = {}
        # to sort by peak time first build a list of all peak times
        for i in range (0, self.numComponents):
            data = self.getDetectorData(i)
            randomEffect = np.random.normal() * 0.02 + 0.98
            peaks[i] = np.argmax(data,axis=1)[1]/100.0 * randomEffect
        
        # sort by peak time.  This gives a tuple for some reason even though picks is a dict
        sorted_peaks = sorted(peaks.items(), key=lambda x: x[1])
        # then iterate through them
        
        for peak in sorted_peaks:
            # peak is a tuple (i, peakTime)
            i = peak[0]
            peakTime = peak[1]
            data = self.getDetectorData(i)
            randomEffect = np.random.normal() * 0.02 + 0.98
            peakHeight = np.max(data) * randomEffect
            peakArea = np.sum(data,axis=1)[1] * randomEffect
            #peakTime = (np.argmax(data,axis=1)[1]/100.0) * randomEffect
            print (f"{peakTime:^20.2f}{peakHeight:^20.2f}{peakArea:^20.2f}")

In [7]:
data = Chromatograph(A=10,B=90)

In [4]:
# initialize a new instance of Chromatograph
chromatograph = Chromatograph()

# setup figure
fig = plt.figure(figsize=[8,6])

# three subplots, one for each compound
ax1 = fig.add_subplot(311)
ax2 = fig.add_subplot(312)
ax3 = fig.add_subplot(313)

ax1.title.set_text('Video of Analyte in the Column')
ax2.title.set_text('Graph:  Amount of Analyte in Mobile Phase')
ax3.set(ylabel='mAu')
ax3.title.set_text('Detector Output')

# initialize plotting stuff
ax2.axis('off')
ax1.axis('off')
plt.close()
lines = []
detectorLines = [] 
plotlays, plotcols = [3], ["black","red","green"]
line, = ax2.plot([], [], lw=3)
detector, = ax3.plot([],[],lw=3)
ims = []

for index in range(3):
    lobj = ax2.plot([],[],lw=2,color=plotcols[index])[0]
    lines.append(lobj)
    detObj = ax3.plot([],[],lw=2,color=plotcols[index])[0]
    detectorLines.append(detObj)
    
# initialize a run

def setupAnimation():
    # set the absorptivities so that the peaks all doen't have the same height
    # roughly corresponds with past data
    epsilon=[1000,750,2000]
    
    # setup mobile phase concentrations, flow rate, etc
    chromatograph.A = A.value
    chromatograph.B = B.value
    chromatograph.epsilon = epsilon
    chromatograph.flowRate = flowRate.value
    chromatograph.runTime = runTime.value
    
    # shh don't tell the students but an 810 mg excedrin tablet is 250 mg aspirin (31%), 250 mg acetaminophen (31%), 65 mg caffeine(8%), and 30% other stuff
    chromatograph.initialValues = [
        aspirin.value + .3086 * excedrin.value,
        acetaminophen.value + .3086 * excedrin.value,
        caffeine.value + .0802 * excedrin.value]
    
    # run the entire simulation to get the max y scale for plotting
    
    # run 10 steps of the simulation and get the max concentration in the mobile phase to set the y-axis max
    chromatograph.reset()
    ax2.set_xlim(0,chromatograph.numPlates)
    chromatograph.nextStep(10)
    ax2.set_ylim(0,chromatograph.maxMobilePhase())
    ax3.set_xlim(0,runTime.value)
    
    # run the rest of the simulation to get the max detector y value
    chromatograph.nextStep(runTime.value*100 - 10)
    ax3.set_ylim(0,chromatograph.maxDetector())
    
    # blank out the image frames and reset to step 0
    ims = []
    chromatograph.reset()

# init is called to start the animation, it just zeroes everything
def init():
    chromatograph.reset()
    for line in lines:
        line.set_data([],[])
    return lines

# animate is called for each frame in the animation
def animate(i):
    # set the data in the mobile phase graph
    for component,line in enumerate(lines):
        line.set_data(chromatograph.getData(component)) # set data for each line separately. 
    # set the data in the dector graph
    for component,detector in enumerate(detectorLines):
        detector.set_data(chromatograph.getDetectorData(component))
        
    # combine all the components in the mobile phase for the column view
    x=chromatograph.getMobilePhase(0)
    y=chromatograph.getMobilePhase(1)
    z=chromatograph.getMobilePhase(2)
    total = x+y+z
    # duplicate the components 10 times to fake a 2-D cross section of the column
    column = np.stack((total,total,total,total,total,total,total,total,total,total))    
    
    # save the image as a frame in the ims frame array
    im = ax1.imshow(column, animated=True, aspect='auto')
    ims.append([im])
    
    # advance the simulation 10 steps (plates)
    chromatograph.nextStep(10)
    return line,
        
        
def startSeparation(_):
    # the animation can be disabled by showAnimation.  It's very slow
    if showAnimation.value:
        print ("Starting Run... Please wait about 2 minutes and then click Play on the video that appears below.")
    else:
        print ("Starting Run, graphics disabled")
        
    # initialize
    setupAnimation()
    # print out mobile phase concentration, flow rate, etc
    chromatograph.printSettings()
    
    # print out injected concentrations
    print (f"aspirin: {aspirin.value} μg/mL    acetaminophen: {acetaminophen.value} μg/mL")
    print (f"caffeine: {caffeine.value} μg/mL    excedrin: {excedrin.value} μg/mL")
    
    # do the animation I used time.time() to see how long it took, it's very slow (2 minutes)
    # the math of the simulation itself only takes a few seconds, but assembling the frames
    # into a movie is slow
    
    if showAnimation.value:
        start = time.time()
        anim = animation.FuncAnimation(fig, animate, init_func=init,
                                   frames=int(runTime.value*10), interval=200, blit=True,repeat_delay=1000)
      #  print(time.time() - start)
        start = time.time()

        anim2 = animation.ArtistAnimation(fig, ims, interval=200, blit=True, repeat_delay=1000)
     #   print(time.time() - start)
        start = time.time()

        # you could save it as a file if you wanted.
          # rc('animation', html='html5')
          #  anim.to_jshtml()

        # jshtml makes it a javascript HTML5 animation
        rc('animation', html='jshtml')
        display(anim)
    else:
        # if you're not animating then just run the whole simulation
        chromatograph.nextStep(runTime.value*100)
        
    chromatograph.printPeakTable()
    print ()
  #  print(time.time() - start)


# setup the widgets for input (mobile phase composition, flow rate, run time, sample concentrations)
style = {'description_width': '200px','width':'300px'}
#style = {'width':'400px'}
A = widgets.IntText(
       value='0',
       description='Water %', style = style)
B = widgets.IntText(
       value='100',
       description='Methanol %', style = style )
flowRate = widgets.FloatText(
       value='1.0',
       description='Flow Rate / mL/min', style = style )
runTime = widgets.IntText(
       value='10',
       description='Run time / min',  style = style)


aspirin = widgets.FloatText(value='100.0',description='[aspirin] / μg/mL',  style = style)
acetaminophen = widgets.FloatText(value='100.0',description='[acetaminophen] / μg/mL',  style = style)
caffeine = widgets.FloatText(value='100.0',description='[caffeine] / μg/mL',  style = style)
excedrin = widgets.FloatText(value='0.0',description='[excedrin] / μg/mL',  style = style)

# this is the button to click to start
startButton = widgets.Button(description='Start Analysis')
startButton.on_click(startSeparation)
showAnimation = widgets.Checkbox(value=True,description='Show Animation (uncheck to speed up)',disabled=False,indent=False)

# arrange the boxes and buttons in a grid.
box = widgets.VBox([
    widgets.HBox([A,B]),
    widgets.HBox([flowRate,runTime]),
    widgets.HBox([aspirin,acetaminophen]),
    widgets.HBox([caffeine,excedrin]),
    widgets.HBox([startButton,showAnimation])])
box